## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]


In [4]:
# Get values from the Attrition column
y_df["Attrition"].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [5]:
# Get values from the Department column
y_df["Department"].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [6]:
# Create a list of at least 10 column names to use as X data
column_list = ["Education", "Age", "DistanceFromHome", "JobSatisfaction", "OverTime", "StockOptionLevel", "WorkLifeBalance", "YearsAtCompany", "YearsSinceLastPromotion", "NumCompaniesWorked"]

# Create X_df using your selected columns
X_df = attrition_df[column_list]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [7]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

random_state = 42

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=random_state)

In [8]:
# Convert your X data to numeric data types however you see fit

# Get the unique values of the OverTime column
X_train["OverTime"].value_counts()

OverTime
No     780
Yes    322
Name: count, dtype: int64

In [9]:
# Convert the OverTime column to a binary column
from sklearn.preprocessing import LabelEncoder

# Create the encoder and fit it to the data
overtime_encoder = LabelEncoder().fit(X_train["OverTime"])

# Transform train and test
X_train["OverTime_Encoded"] = overtime_encoder.transform(X_train["OverTime"])
X_test["OverTime_Encoded"] = overtime_encoder.transform(X_test["OverTime"])

# Drop the original OverTime column from the X data
X_train.drop("OverTime", axis=1, inplace=True)
X_test.drop("OverTime", axis=1, inplace=True)

In [10]:
# Check training columns
X_train.columns

Index(['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
       'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'NumCompaniesWorked', 'OverTime_Encoded'],
      dtype='object')

In [11]:
# Check testing columns
X_test.columns

Index(['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
       'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'NumCompaniesWorked', 'OverTime_Encoded'],
      dtype='object')

In [12]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)  

In [13]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[["Department"]])
department_test_encoded = department_encoder.transform(y_test[["Department"]])

department_train_encoded


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [14]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[["Attrition"]])   

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[["Attrition"]])
attrition_test_encoded = attrition_encoder.transform(y_test[["Attrition"]])

attrition_train_encoded


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [15]:
X_train_scaled.shape

(1102, 10)

In [16]:
X_test_scaled.shape

(368, 10)

## Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns), name="input_layer")

# Create at least two shared layers
shared1 = layers.Dense(units=64, activation="relu", name="shared1")(input_layer)
shared2 = layers.Dense(units=128, activation="relu", name="shared2")(shared1)

In [18]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=32, activation="relu", name="department_hidden")(shared2)

# Create the output layer
department_output = layers.Dense(units=3, activation="softmax", name="department_output")(department_hidden)


In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=32, activation="relu", name="attrition_hidden")(shared2)

# Create the output layer
attrition_output = layers.Dense(units=2, activation="softmax", name="attrition_output")(attrition_hidden)


In [20]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer="adam", 
              loss={"department_output": "categorical_crossentropy", "attrition_output": "categorical_crossentropy"}, 
              metrics={"department_output": "accuracy","attrition_output": "accuracy"})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 10)]         0           []                               
                                                                                                  
 shared1 (Dense)                (None, 64)           704         ['input_layer[0][0]']            
                                                                                                  
 shared2 (Dense)                (None, 128)          8320        ['shared1[0][0]']                
                                                                                                  
 department_hidden (Dense)      (None, 32)           4128        ['shared2[0][0]']                
                                                                                              

In [24]:
# Train the model
model.fit(X_train_scaled, 
          {"department_output": department_train_encoded, "attrition_output": attrition_train_encoded}, 
          epochs=100,
          verbose=2)


Epoch 1/100
35/35 - 0s - loss: 0.0612 - department_output_loss: 0.0489 - attrition_output_loss: 0.0122 - department_output_accuracy: 0.9909 - attrition_output_accuracy: 0.9982 - 28ms/epoch - 786us/step
Epoch 2/100
35/35 - 0s - loss: 0.0567 - department_output_loss: 0.0387 - attrition_output_loss: 0.0180 - department_output_accuracy: 0.9964 - attrition_output_accuracy: 0.9964 - 24ms/epoch - 699us/step
Epoch 3/100
35/35 - 0s - loss: 0.0547 - department_output_loss: 0.0414 - attrition_output_loss: 0.0133 - department_output_accuracy: 0.9936 - attrition_output_accuracy: 0.9982 - 25ms/epoch - 707us/step
Epoch 4/100
35/35 - 0s - loss: 0.0618 - department_output_loss: 0.0513 - attrition_output_loss: 0.0106 - department_output_accuracy: 0.9873 - attrition_output_accuracy: 0.9973 - 24ms/epoch - 698us/step
Epoch 5/100
35/35 - 0s - loss: 0.0979 - department_output_loss: 0.0823 - attrition_output_loss: 0.0156 - department_output_accuracy: 0.9719 - attrition_output_accuracy: 0.9973 - 25ms/epoch - 7

In [25]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled,{"department_output": department_test_encoded, "attrition_output": attrition_test_encoded})

test_results

12/12 [==============================] - 0s 773us/step - loss: 5.6161 - department_output_loss: 4.1487 - attrition_output_loss: 1.4674 - department_output_accuracy: 0.5163 - attrition_output_accuracy: 0.8207


[5.616080284118652,
 4.148706912994385,
 1.467373251914978,
 0.5163043737411499,
 0.820652186870575]

In [26]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {test_results[3]}")
print(f"Attrition predictions accuracy: {test_results[4]}")

Department predictions accuracy: 0.5163043737411499
Attrition predictions accuracy: 0.820652186870575


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. If looking at attrition only predictions, accuracy rate is approximately 82.07%.  This would be a arguably good but not great rate. It would be worth testing additional models and metrics to see if the accuracy rate could be improved. However, department accuracy is approximately 51.63% which is little better than 50-50. This would not be a good metric for department accuracy. Using this model to evaluate both attrition and department predictions, it would need to have high accurary rates for both categories. In a combined evaluation, accuracy is not the best metric on this data.
2. The activation function was chosed based on the values for each column.
Department has 3 values and was a multi class column. Softmax activation was chosen for multi class.
Attrition had 2 values and was a binary class column. Sigmoid activation was chosen for binary.
3. 
* The original columns selected could be altered. Different or more columns could be chosen. Correlations and metrics could be ran to determine the most relevant columns.
* Additional shared layers could be created. The steps above only had two shared layers.
* Additional hidden layers for both department and attrition could be created. The steps above only had one hidden layer.
* The hidden layers for department and attrition could be branched at different layers. For example, currently attrition hidden layer is using the shared layer 2. A third shared layer could be added and the department could be pointed to the shared layer 3. An additional processing layer may increase the accuracy.
* The parameters for the shared and hidden layers could be adjusted.